# 1. Jupyterlab

## 1.1. Setup

1. Choose the `Python [conda env:.venv]` from the menu in the top right corner of this page.

2. The `dsdk` and `cfgenvy` modules are already installed into the conda venv, along with any dependencies added to requirement.txt SO LONG AS jupyterlab has been started with `docker-compose up --build jupyterlab &`. The docker container image build for jupyterlab will cache build steps applying `./jupyterlab/environment.yaml` with from modules anaconda and conda forge, and will apply `./requirements.txt` last with modules from pypi, or specifically pulled from public git repositories --this is a way to install your own data science helper modules like dsdk--. Only the first run of the jupyterlab container image build will be slow so long as environment.yaml is not changed.

2. If there is a predict python module developed with this project, it may also be installed into the conda env, but this step is optional and may not be useful: --see setup.py--:

    a. Production vs development mode: `pip install ...` vs. `pip install -e ...`. Development mode builds symlinks between site-packages and the code in src. Reinstall is not required after changes, but there other are limitations.

    b. Sparce vs with additional dependencies: `pip install .` vs. `pip install .[all]`. Additional dependencies include lint and test tools.

In [1]:
!pip install -e ".[all]"

Obtaining file:///tmp
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/pennsignals/dsdk.git (to revision 1.4.7) to ./pip-install-mfj6y7wx/dsdk_e65ce6a8e72145b58d714e9b238e1444
  Running command git clone --filter=blob:none -q https://github.com/pennsignals/dsdk.git /tmp/pip-install-mfj6y7wx/dsdk_e65ce6a8e72145b58d714e9b238e1444
  Resolved https://github.com/pennsignals/dsdk.git to commit d6513c8053a2017281aa89c712b9b41bb685ea61
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 191 kB 1.8 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 280 kB 3.5 MB/s       

## 1.1.1 Setup (continued):

3. Reload the conda env after each production mode install and once after a development mode install. Use the restart button next to the run and stop buttons in the toolbar.

4. Check outdated packages for any unexpected suprises.

In [2]:
!pip list --outdated

Package            Version Latest Type
------------------ ------- ------ -----
anyio              3.4.0   3.5.0  wheel
charset-normalizer 2.0.9   2.0.10 wheel
GitPython          3.1.25  3.1.26 wheel
idna               3.1     3.3    wheel
ipykernel          6.6.1   6.7.0  wheel
ipython            7.31.0  8.0.0  wheel
json5              0.9.5   0.9.6  wheel
jsonschema         4.3.3   4.4.0  wheel
jupyter-server     1.13.1  1.13.2 wheel
jupyterlab         3.2.5   3.2.8  wheel
jupyterlab-git     0.34.0  0.34.1 wheel
jupyterlab-server  2.10.2  2.10.3 wheel
mistune            0.8.4   2.0.2  wheel
nbclassic          0.3.4   0.3.5  wheel
nbclient           0.5.9   0.5.10 wheel
notebook           6.4.6   6.4.7  wheel
numexpr            2.8.0   2.8.1  wheel
Pillow             8.4.0   9.0.0  wheel
platformdirs       2.3.0   2.4.1  wheel
psycopg2           2.9.2   2.9.3  sdist
Pygments           2.11.1  2.11.2 wheel
PyQt5              5.12.3  5.15.6 wheel
PyQt5_sip          4.19.18 12.9.0 wheel
P

## 2. Manage Configuration & Environment

### 2.1. Files

Because deserializing objects is less error prone than (re-)configuring previously existing python objects, use cfgenvy to load and dump yaml as configuration. Merge environment variable files into yaml configuration during deserialization, and keep your secrets separate and safe.

Secrets as well as differences among deployment environments are placed in .env files: `./predict/secrets/`.

Configurations are placed in .yaml files: `./predict/local/`.

These directories have .gitignore protection from accidental inclusion in version control. See the `./predict/secrets/.gitignore` and `./predict/local/.gitignore` files for file names that *ARE* included in version control.

In [3]:
import osfrom dsdk import Asset, Mssql, Postgres
from cfgenvy import yaml_loads, yaml_dumps, Parser, YamlMapping
from typing import Any, Dict, List, Optional

Some file names and a parser:

In [4]:
os.chdir('/tmp')"
config_file = "./predict/local/notebook.example.yaml"
env_file = "./predict/secrets/notebook.example.env"

parser = Parser()

Service names are resolved to host ip addresses by docker DNS as listed in docker-compose.override.yaml, and later by consul DNS in production. Use service names when possible instead of ip addresses. Even names for external services external like clarity, and epic can be registered in consul DNS to keep ip addresses out of configuration files.

Here the MSSQL_HOST and POSTGRES_HOST are service names:

In [5]:
envs_str = """
EPIC_COOKIE=cookie
MSSQL_DATABASE=clarity
MSSQL_HOST=mssql
MSSQL_PASSWORD=password
MSSQL_PORT=1433
MSSQL_USERNAME=username
POSTGRES_DATABASE=test
POSTGRES_HOST=postgres
POSTGRES_PASSWORD=password
POSTGRES_PORT=5432
POSTGRES_SCHEMA=test
POSTGRES_USERNAME=postgres"""

In [6]:
with open(env_file, "w", encoding="utf-8") as fout:
    fout.write(envs_str)

In [7]:
cfgs_str = """
elixhauser:
  key1: val1
  key2: val2
  key3: val3
mssql: !mssql
  database: ${MSSQL_DATABASE}
  host: ${MSSQL_HOST}
  password: ${MSSQL_PASSWORD}
  port: ${MSSQL_PORT}
  schema: test
  sql: !asset
    path: ./predict/sql/mssql
    ext: .sql
  username: ${MSSQL_USERNAME}
postgres: !postgres
  database: ${POSTGRES_DATABASE}
  host: ${POSTGRES_HOST}
  password: ${POSTGRES_PASSWORD}
  port: ${POSTGRES_PORT}
  schema: test
  sql: !asset
    path: ./predict/sql/postgres
    ext: .sql
  username: ${POSTGRES_USERNAME}
stages:
- first
- second
- third"""

In [8]:
with open(config_file, "w", encoding="utf-8") as fout:
    fout.write(cfgs_str)

Register classes as yaml types so they may be deserialized as instaces of python classes:

In [9]:
Mssql.as_yaml_type()
Postgres.as_yaml_type()

cfg = parser.load(
    config_file=config_file,
    env_file=env_file,
)

print(f"type(cfg): {type(cfg)}")
print(f"type(cfg['elixhauser']: {type(cfg['elixhauser'])}")
print(f"type(cfg['postgres']: {type(cfg['postgres'])}")
print(f"type(cfg['postgres'].sql: {type(cfg['postgres'].sql)}")
print(f"type(cfg['stages']): {type(cfg['stages'])}")

type(cfg): <class 'dict'>
type(cfg['elixhauser']: <class 'dict'>
type(cfg['postgres']: <class 'dsdk.postgres.Persistor'>
type(cfg['postgres'].sql: <class 'dsdk.asset.Asset'>
type(cfg['stages']): <class 'list'>


Create and register a class to provide better validation for confguration and by ensuring that the configuration file is not mismatched, use explicit yaml `!<type>` and a clss. Unlike a python dictionary, unexpected or missing keywords will raise early exceptions.

In [10]:
with open(config_file, "w", encoding="utf-8") as fout:
    fout.write("!cfg" + cfgs_str)

In [11]:
class Cfg(YamlMapping):

    YAML = '!cfg'
    
    def __init__(
        self,
        *,
        elixhauser: Dict[str, str],
        mssql: Mssql,
        postgres: Postgres,
        stages: List,
    ):
        self.elixhauser = elixhauser
        self.mssql = mssql
        self.postgres = postgres
        self.stages = stages
    
    def as_yaml(self) -> Dict[str, Any]:
        """As yaml."""
        return {
            "elixhauser": self.elixhauser,
            "mssql": self.mssql,
            "postgres": self.postgres,
            "stages": self.stages,
        }

In [12]:
Cfg.as_yaml_type()

cfg = parser.load(
    config_file=config_file,
    env_file=env_file,
)

print(f"type(cfg): {type(cfg)}")
print(f"type(cfg.elixhauser): {type(cfg.elixhauser)}")
print(f"type(cfg.postgres): {type(cfg.postgres)}")
print(f"type(cfg.postgres.sql): {type(cfg.postgres.sql)}")
print(f"type(cfg.stages): {type(cfg.stages)}")

type(cfg): <class '__main__.Cfg'>
type(cfg.elixhauser): <class 'dict'>
type(cfg.postgres): <class 'dsdk.postgres.Persistor'>
type(cfg.postgres.sql): <class 'dsdk.asset.Asset'>
type(cfg.stages): <class 'list'>


In [13]:
postgres = cfg.postgres

Debug the final merged configuration:

In [14]:
print(yaml_dumps(postgres))

!postgres
database: test
host: postgres
password: password
port: '5432'
schema: test
sql: !asset
  ext: .sql
  path: ./predict/sql/postgres
username: postgres



## 3. Check Database Connectivity

In [15]:
with postgres.rollback() as cursor:
    cursor.execute("""select 'Very database, much wow!' as doge""")
    rows = cursor.fetchall()
    print(rows[0])

('Very database, much wow!',)


## 4. Manage SQL & Other Text Assets

Assets loads text files from disk. Unlike SQL embedded in python strings, SQL syntax highlighting may be available in text editor. The python placeholders expected by psycopg2 and pymssql will still be marked as errors.

In [16]:
print(postgres.sql.predictions.gold)

select
    score
from
    predictions
where
    run_id = %(run_id)s
order by
    id desc;



# 5. Rethink SQL

In [17]:
keys = {
    "cohort": ('00001', '00002', '00003'),
    "conditions" : ('sleepy', 'happy', 'grumpy'),
}

parameters = {
    "dry_run": 0,
    "cohort_begin": '2021-05-05',
    "cohort_end": '2021-05-06',
}

## 5.1. Prefer `with` over `in (?, ...)`:

Avoid `in` for more than a few elements:

`select * from patients where id in ('00001', '00002', '00003', ...);`

Unfortunately, the execution plan renders `in` similar to multiple `or`:

`select * from patients where id = '00001' or id = '00002' or id = '00003' ...;`

The performance is terrible. The database has limits on the number of elements that may be included using `in (?, ...)`. Fundamentally, the database does not treat `in` like a table with a single column, in part because the column data type is not known. Client languages like python typically only have data types that approximately match the database's data types. For example the pymssql driver passes all python strings to mssql as `nvarchar` literals ('n' is not a typo). Each element is coherced to the most permissive data type during comparison. This implicit, permissive casting and cohersion prevents indices from being used.

Use `with` instead and `cast` the column to the appropriate data type.

### 5.1.1. Example:

An easy example in templated sql for python and dsdk looks like this:

In [18]:
query_5_1_1 = '''
with cohort as (
    select cast(null as varchar(8)) as id -- data type is on the cohort.id column, not just this first row
    {cohort}
)
select
    id
from
    cohort
where
    id is not null;'''

with postgres.rollback() as cur:
    df = postgres.df_from_query_by_keys(cur, query_5_1_1, keys, parameters)
    
df

,id
0,00001
1,00002
2,00003


### 5.1.2. Example:

A more useful example using dsdk looks like this:

In [19]:
query_5_1_2 = '''
with args as (
    select
        cast(%(cohort_begin)s as timestamptz) as cohort_begin,
        cast(%(cohort_end)s as timestamptz) as cohort_end
), cohort as (
    select cast(null as varchar(8)) as id
    {cohort}
), conditions as (
    select cast(null as varchar(16)) as name
    {conditions}
)
select
    cohort_begin,
    cohort_end,
    id,
    name
from
    args
    join cohort
        on id is not null
    join conditions
        on name is not null;
'''

with postgres.rollback() as cur:
    df = postgres.df_from_query_by_keys(cur, query_5_1_2, keys, parameters)
    
df

,cohort_begin,cohort_end,id,name
0,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00001,sleepy
1,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00001,happy
2,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00001,grumpy
3,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00002,sleepy
4,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00002,happy
5,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00002,grumpy
6,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00003,sleepy
7,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00003,happy
8,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00003,grumpy


### 5.1.3. Example

Implementation of dsdk for df_from_query_by_keys uses `union all select` implementation. This formulation avoids item limits as well as comma counting of `insert (...) values (...), ...`. Unlike `in` and `insert (...) values (...), ...` it also results in perfectly valid sql even when the cohort or conditions lists empty, because the empty lists render as code while retaining the column data type(s) using the "null row".

Unwind the sequences and replace the placeholders in pgadmin, DBeaver, Data Grip, and Microsort Sql Server Management Studio to test and explain your queries:

In [20]:
query_5_1_3 = '''
with args as (
    select
        cast('2021-05-05' as timestamptz) as cohort_begin,
        cast('2021-05-06' as timestamptz) as cohort_end
), cohort as (
    select cast(null as varchar) as id
    union all select '00001'
    union all select '00002'
    union all select '00003'
), conditions as (
    select cast(null as varchar) as name
    union all select 'happy'
    union all select 'sleepy'
    union all select 'grumpy'
)
select
    cohort_begin,
    cohort_end,
    id,
    name
from
    args
    join cohort
        on id is not null
    join conditions
        on name is not null;'''

from pandas import DataFrame

with postgres.rollback() as cur:
    cur.execute(query_5_1_3)
    rows = cur.fetchall()
    df = DataFrame(rows)
    columns = (each[0] for each in cur.description)
    df.columns = columns

df

,cohort_begin,cohort_end,id,name
0,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00001,happy
1,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00001,sleepy
2,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00001,grumpy
3,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00002,happy
4,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00002,sleepy
5,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00002,grumpy
6,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00003,happy
7,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00003,sleepy
8,2021-05-05 00:00:00+00:00,2021-05-06 00:00:00+00:00,00003,grumpy


## 5.2 Use dry run to fail early

Make the database do more work for you. This includes validating some syntax and all permission on the service accounts BEFORE passing actual useful data to the database.

In [21]:
query_5_2 = '''
with vars as (
    select
        cast(coalesce(%(dry_run)s, 1) as int) as dry_run,
        cast(%(cohort_begin)s as timestamptz) as cohort_begin,
        cast(%(cohort_end)s as timestamptz) as cohort_end
), cohort as (
    select cast(null as varchar) as id
    {cohort}
)
select
    no_such_table.*
from
    vars as v
    join cohort as c
        on v.dry_run = 0
        and c.id is not null;'''

In [22]:
postgres.dry_run_query(query_5_2, parameters)

UndefinedTable: missing FROM-clause entry for table "no_such_table"
LINE 12:     no_such_table.*
             ^


Persistors can dry run all sql queries in an asset if all parameters are provided. All queries must be written to select, insert, update or delete no data when dry_run is 1, but must do by producing empty data sets for insert, update, and delete instead of exiting early.

Typically, this means using a `with` clause to build a data set for insert, update or delete and performing a join on `dry_run = 0` that knocks out all rows from the data manipulation operators.

More examples to come, and all queries in the postgres persistor asset must be revised for dry_run compatibility.

More examples to come on when to add unused tables to aquire indices.

More examples to come on sql performance profiling and explain.

More example on when using temp tables may be an advantage, and the impact on readability, maintainability, and testing.

In [ ]:
postgres.dry_run({})